<a href="https://colab.research.google.com/github/urieliram/statistical/blob/main/Tarea12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error,mean_squared_error,mean_absolute_percentage_error
from sklearn import svm
from sklearn.svm import LinearSVR


In [16]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error

Ahora, cargamos los datos de prueba en los arreglos `X_train` y `y_train`:

In [17]:
df    = pd.read_csv('demanda.csv')

## Procesamos datos de entrenamiento 
dftrain = df.loc[df.iloc[:,20].isin(['T'])] ## 'T' = training set
y_train = dftrain['Y'].to_numpy()  
X_train = dftrain[['X0','X1','X2','X3','X4','X5','X6','X7','X8','X9','X10']].to_numpy() ## Regresores

## Procesamos datos de prueba 
dftest = df.loc[df.iloc[:,20].isin(['F'])] ## 'F' = test set
y_test = dftest['Y'].to_numpy()
X_test = dftest[['X0','X1','X2','X3','X4','X5','X6','X7','X8','X9','X10']].to_numpy() ## Regresores

In [18]:
def trunc(values, decs=0):
    return np.trunc(values*10**decs)/(10**decs)

In [19]:
model = LinearRegression()
model.fit(X_train, y_train)
r_sq = model.score(X_test, y_test)
print('coefficient of determination:', r_sq)
print('intercept:', model.intercept_)
print('slope:', model.coef_)
y_pred = model.predict(X_test)
mae_rlm  = trunc(mean_absolute_error(y_test,y_pred),4)
mse_rlm  = trunc(mean_squared_error(y_test,y_pred),4)
mape_rlm = trunc(mean_absolute_percentage_error(y_test,y_pred),4)

coefficient of determination: 0.7459708965464427
intercept: -665.3444836197305
slope: [ 0.          0.28749465  0.29204618  0.25861088 -0.04431544 -0.14208937
 -0.02613895  0.18289727  0.33580077  0.06905434 -0.12973508]


In [22]:
mae_svm = []; mse_svm = []; mape_svm = []; 
Clist = [];
perc_within_eps_list = []
epsilon = 100
for c in range(40, 55, 1):
    Clist.append(c)
    model = svm.SVR(kernel='rbf', C=c, epsilon=epsilon)
    model.fit(X_train, y_train)
    r_sq = model.score(X_test, y_test)
    print('coefficient of determination:', r_sq)
    
    y_pred = model.predict(X_test)
    mae_svm.append(trunc(mean_absolute_error(y_test,y_pred),4))
    mse_svm.append(trunc(mean_squared_error(y_test,y_pred),4))
    mape_svm.append(trunc(mean_absolute_percentage_error(y_test,y_pred),4))
    
    perc_within_eps = 100*np.sum(abs(y_test-y_pred) <= epsilon) / len(y_test)
    perc_within_eps_list.append(perc_within_eps)
    print(c,perc_within_eps )

coefficient of determination: 0.7813628808809677
40 39.583333333333336
coefficient of determination: 0.7815529379004028
41 40.27777777777778
coefficient of determination: 0.7833506926110352
42 41.666666666666664
coefficient of determination: 0.7855049683233903
43 41.31944444444444
coefficient of determination: 0.7877408223154592
44 41.666666666666664
coefficient of determination: 0.7878461304614096
45 42.013888888888886
coefficient of determination: 0.7887290305729372
46 42.361111111111114
coefficient of determination: 0.7912038736360554
47 43.05555555555556
coefficient of determination: 0.7909423850105974
48 44.44444444444444
coefficient of determination: 0.7921037221277013
49 44.44444444444444
coefficient of determination: 0.7923868611918274
50 44.09722222222222
coefficient of determination: 0.7935270330680637
51 44.791666666666664
coefficient of determination: 0.7934276878423224
52 44.791666666666664
coefficient of determination: 0.7949512795985678
53 45.138888888888886
coefficient 

In [23]:
print('| REGRESIÓN      | C              | MAE            | MSD            | MAPE         |')
print('| :------------- | :------------- | -------------: | -------------: |-------------:|')
print('|    lineal      |         ',       0,'    |    ',str(mae_rlm),'  |    ',str(mse_rlm),'    |    ',str(mape_rlm),'|' )
for i in range(len(Clist)):
    print('|     SVM        |       ',  Clist[i]  ,'    |    ',str(mae_svm[i]),'   |    ',
          str(mse_svm[i]),'    |    ',str(mape_svm[i]),'|','    |    ',str(perc_within_eps_list[i]),'|',)


| REGRESIÓN      | C              | MAE            | MSD            | MAPE         |
| :------------- | :------------- | -------------: | -------------: |-------------:|
|    lineal      |          0     |     167.1343   |     48064.1398     |     0.0192 |
|     SVM        |        40     |     158.8235    |     41367.721     |     0.0179 |     |     39.583333333333336 |
|     SVM        |        41     |     158.2783    |     41331.7608     |     0.0178 |     |     40.27777777777778 |
|     SVM        |        42     |     157.3381    |     40991.6126     |     0.0177 |     |     41.666666666666664 |
|     SVM        |        43     |     156.2612    |     40584.0081     |     0.0176 |     |     41.31944444444444 |
|     SVM        |        44     |     155.2193    |     40160.9684     |     0.0175 |     |     41.666666666666664 |
|     SVM        |        45     |     155.0974    |     40141.0433     |     0.0174 |     |     42.013888888888886 |
|     SVM        |        46     |    

In [14]:
#https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
#https://github.com/tomsharp/SVR/blob/master/SVR.ipynb
parameters = {'kernel':('linear', 'rbf'), 'C':Clist,} #'epsilon': np.linspace(1, 500)
model = svm.SVR()
clf   = GridSearchCV(model, parameters,scoring='neg_mean_absolute_error', cv=5)
clf.fit(X_train, y_train)
sorted(clf.cv_results_.keys())
print(clf.best_estimator_)
model = clf.best_estimator_
model.fit(X_train, y_train)

r_sq = model.score(X_test, y_test)
print('coefficient of determination:', r_sq)
    
y_pred = model.predict(X_test)

SVR(C=54)
coefficient of determination: 0.8002363165218531


[https://towardsdatascience.com/an-introduction-to-support-vector-regression-svr-a3ebc1672c2](https://towardsdatascience.com/an-introduction-to-support-vector-regression-svr-a3ebc1672c2)

[Grid Search over C and Epsilon](https://github.com/tomsharp/SVR/blob/master/SVR.ipynb)

https://scikit-learn.org/stable/modules/svm.html

https://scikit-learn.org/stable/auto_examples/svm/plot_svm_regression.html

https://medium.com/pursuitnotes/support-vector-regression-in-6-steps-with-python-c4569acd062d